In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
# read in pitch category csvs

fb = pd.read_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\fastball_ready.csv")
bb = pd.read_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\breakingball_ready.csv")
os = pd.read_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\offspeed_ready.csv")

In [15]:
# combine all 3 dfs

all_pitches = pd.concat([fb, bb, os], ignore_index=True)

In [16]:
# pare columns, sort by stuff plus, format index

all_ready = all_pitches[['player_name', 'pitch_type', 'stuff_plus', 'count']].reset_index(drop=True)
all_ready.sort_values(by='stuff_plus', ascending=False, inplace=True)
all_ready = all_ready.reset_index(drop=True)
all_ready.index = all_ready.index + 1

In [17]:
# save to y drive

all_ready.to_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\all_ready.csv")

In [18]:
# groupby pitch count per player to get total pitch count, filter for players that threw at least 300 pitches this season

pitch_counts = all_pitches.groupby('player_name')['count'].sum().reset_index()
pitch_counts.rename(columns={'count': 'total_count'}, inplace=True)
pitch_counts = pitch_counts[pitch_counts['total_count'] >= 300]

In [19]:
# merge all pitches and pitch counts

arsenals = pd.merge(all_pitches, pitch_counts, on='player_name', how='left')

# divide count by total count to get usage rate

arsenals['usage'] = arsenals['count'] / arsenals['total_count']

# get the product of usage rate and stuff plus for each pitch and sum per player to get arsenal score (weighted stuff plus avg)

arsenals['arsenal_score'] = arsenals['stuff_plus'] * arsenals['usage']
arsenal_score = arsenals.groupby('player_name')['arsenal_score'].sum().reset_index()

# formatting

arsenal_score['arsenal_score'] = arsenal_score['arsenal_score'].astype(int)
arsenal_score.sort_values(by='arsenal_score', ascending=False, inplace=True)
arsenal_score = arsenal_score.reset_index(drop=True)
arsenal_score.index = arsenal_score.index + 1

In [20]:
# select top 25 rows

top_25_rows = arsenal_score[['player_name', 'arsenal_score']].head(25)

# plotting rankings as a table

fig, ax = plt.subplots(figsize=(5, 6))
ax.axis('tight')
ax.axis('off')
ax.set_title('Top 25 Offspeed Stuff+')
table = ax.table(cellText=top_25_rows.values,
                 colLabels=top_25_rows.columns,
                 loc='center',
                 cellLoc='center',
                 colLoc='center',
                 colWidths=[0.4, 0.2])
plt.tight_layout()

# save fig and close

plt.savefig('arsenal_rankings.png', dpi=300)
plt.close()

In [21]:
# save to y drive

arsenal_score.to_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\arsenal_score.csv")

In [22]:
print(all_pitches[all_pitches['player_name'] == 'Spencer Strider'])

      Unnamed: 0      player_name pitch_type  stuff_plus  count
7            573  Spencer Strider         FF         206   1940
1219         565  Spencer Strider         SL          83   1107
1674         232  Spencer Strider         CH          69    237
